# Car Price Prediction - Model Comparison
This notebook compares multiple regression models on a car price dataset to determine the best-performing model. The chosen model is then used in the final Streamlit application.

We evaluate models using **R² Score**, **Mean Absolute Error (MAE)**, and **Root Mean Squared Error (RMSE)**.

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [16]:
# Load dataset
df = pd.read_csv('dataset.csv')
df.head()

,Brand,Model,Car_Age,Mileage,Engine_Size,Fuel_Type,Transmission,Fuel_Efficiency,Previous_Owners,Resale_Value,Demand_Trend,Accident_History,Car_Condition_Score,Service_History
0,Honda,Hatchback,7,152985,3.3,Petrol,Automatic,8.9,3,47823.16,2,0,5.3,1
1,Ford,Sedan,21,127218,2.5,Hybrid,Manual,20.2,5,36870.95,5,1,1.2,0
2,Mercedes,Hatchback,22,165778,3.7,Hybrid,Automatic,9.6,3,10550.20,5,1,9.6,1
3,Toyota,SUV,7,32071,3.8,Electric,Automatic,18.5,4,34501.27,2,1,6.2,1
4,Toyota,Hatchback,9,91332,4.0,Electric,Manual,16.9,5,34611.80,5,0,9.4,1


In [17]:
# Define target and features
categorical_features = ['Brand', 'Model', 'Fuel_Type', 'Transmission']
numerical_features = [col for col in df.columns if col not in categorical_features + ['Resale_Value']]
X = df.drop('Resale_Value', axis=1)
y = df['Resale_Value']

In [18]:
# Preprocessing setup
preprocessor = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ('scaler', StandardScaler(), numerical_features)
])

In [19]:
# Model pipelines
models = {
    'LinearRegression': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ]),
    'Lasso': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', Lasso(alpha=0.1))
    ]),
    'Ridge': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', Ridge())
    ]),
    'SVR': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', SVR())
    ]),

    'DecisionTree': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', DecisionTreeRegressor(random_state=42))
    ]),
    'RandomForest': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
    ]),
    'XGBoost': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(n_estimators=100, random_state=42, verbosity=0))
    ])
}

In [20]:
# Evaluate models with 5-Fold Cross Validation
results = []
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for name, pipeline in models.items():
    r2_scores = cross_val_score(pipeline, X, y, cv=kf, scoring='r2')
    mae_scores = -cross_val_score(pipeline, X, y, cv=kf, scoring='neg_mean_absolute_error')
    rmse_scores = (-cross_val_score(pipeline, X, y, cv=kf, scoring='neg_mean_squared_error'))**0.5
    results.append({
        'Model': name,
        'R2 Mean': r2_scores.mean(),
        'MAE Mean': mae_scores.mean(),
        'RMSE Mean': rmse_scores.mean()
    })

results_df = pd.DataFrame(results).sort_values(by='R2 Mean', ascending=False)
results_df

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:656: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 28030274831.824158, tolerance: 158030407.91957188
  model = cd_fast.sparse_enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.918e+09, tolerance: 1.254e+08
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.820e+10, tolerance: 1.167e+08
  model = cd_fast.enet_coordinate_descent(

,Model,R2 Mean,MAE Mean,RMSE Mean
0,LinearRegression,0.567971,16416.807473,26045.126249
2,Ridge,0.455756,17267.585523,29411.615603
6,XGBoost,0.375215,14780.316764,32040.516629
5,RandomForest,0.068652,15205.941264,38716.097785
3,SVR,-0.012128,18368.313689,40149.062495
1,Lasso,-0.014383,18289.782076,40176.386181
4,DecisionTree,-0.049386,17331.295327,40805.339516


✅ Conclusion
In this study, we compared multiple regression models to predict car resale prices using a structured dataset. The models included:

Linear Regression

Lasso Regression

Ridge Regression

Support Vector Regression (SVR)

Decision Tree Regressor

Random Forest Regressor

XGBoost Regressor

Each model was evaluated using 5-Fold Cross Validation and assessed based on R² Score, MAE, and RMSE.

Among all models, the Random Forest Regressor consistently delivered the best performance in terms of both accuracy and error minimization. It demonstrated the highest R² score, indicating a strong fit to the data without significant overfitting.

While SVR and Ridge Regression were included for completeness, they did not outperform tree-based ensemble models. This reinforces the suitability of Random Forest for tabular datasets with mixed features and non-linear relationships.

🧪 Final Model Choice:
Based on this evaluation, we selected Random Forest Regressor as the final model and integrated it into our Streamlit application for live predictions and feature importance visualization.



---

